In [6]:
!pip install anthropic -q

In [7]:
import json
import os
import logging
import datetime
import anthropic
from anthropic.types import TextBlock, ToolUseBlock

# Initialize the client
anth_api_key = os.getenv('ANTHROPIC_API_KEY')
client = anthropic.Anthropic(api_key=anth_api_key)

In [8]:
# Define the tools, including domain-specific tools and the "think" tool
def get_tools():
    print("Loading tool definitions")
    return [
        # Customer lookup tool
        {
            "name": "get_customer",
            "description": "Gets customer information by customer ID",
            "input_schema": {
                "type": "object",
                "properties": {
                    "customer_id": {
                        "type": "string",
                        "description": "The customer ID"
                    }
                },
                "required": ["customer_id"]
            }
        },
        # Order lookup tool
        {
            "name": "get_order",
            "description": "Gets order information by order ID",
            "input_schema": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The order ID"
                    }
                },
                "required": ["order_id"]
            }
        },
        # Return policy check tool
        {
            "name": "check_return_eligibility",
            "description": "Checks if an item is eligible for return based on product ID and days since purchase",
            "input_schema": {
                "type": "object",
                "properties": {
                    "product_id": {
                        "type": "string",
                        "description": "The product ID"
                    },
                    "days_since_purchase": {
                        "type": "integer",
                        "description": "Number of days since the purchase was made"
                    },
                    "product_condition": {
                        "type": "string",
                        "description": "Condition of the product",
                        "enum": ["new", "used", "damaged"]
                    }
                },
                "required": ["product_id", "days_since_purchase", "product_condition"]
            }
        },
        # Process return tool
        {
            "name": "process_return",
            "description": "Processes a return request",
            "input_schema": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The order ID"
                    },
                    "product_id": {
                        "type": "string",
                        "description": "The product ID"
                    },
                    "reason": {
                        "type": "string",
                        "description": "The reason for return"
                    },
                    "refund_method": {
                        "type": "string",
                        "description": "The method for refund",
                        "enum": ["original_payment", "store_credit", "exchange"]
                    }
                },
                "required": ["order_id", "product_id", "reason", "refund_method"]
            }
        },
        # The "think" tool - simply a scratchpad for Claude's reasoning
        {
            "name": "think",
            "description": """Use this tool to think about something.
            It will not obtain new information or change the database, but just append the thought to the log.
            Use it when complex reasoning or some cache memory is needed.""",
            "input_schema": {
                "type": "object",
                "properties": {
                    "thought": {
                        "type": "string",
                        "description": "A thought to think about."
                    }
                },
                "required": ["thought"]
            }
        }
    ]

# Mock data to simulate database responses
CUSTOMER_DB = {
    "C123456": {
        "name": "Jane Smith",
        "email": "jane.smith@example.com",
        "type": "premium",
        "since": "2023-01-15"
    },
    "C789012": {
        "name": "John Doe",
        "email": "john.doe@example.com",
        "type": "standard",
        "since": "2024-02-20"
    }
}

ORDER_DB = {
    "ORD-9876": {
        "customer_id": "C123456",
        "date": "2025-02-01",
        "items": [
            {"product_id": "P-ELECTRONICS-1001", "name": "Wireless Headphones", "price": 149.99, "quantity": 1},
            {"product_id": "P-CLOTHING-5032", "name": "Premium Cotton T-shirt", "price": 29.99, "quantity": 2}
        ],
        "total": 209.97
    },
    "ORD-5432": {
        "customer_id": "C789012",
        "date": "2025-03-10",
        "items": [
            {"product_id": "P-BOOKS-3021", "name": "Python Programming Guide", "price": 49.99, "quantity": 1}
        ],
        "total": 49.99
    }
}

PRODUCT_RETURN_POLICIES = {
    "P-ELECTRONICS-1001": {"max_days": 30, "conditions": ["new", "used"]},
    "P-CLOTHING-5032": {"max_days": 45, "conditions": ["new"]},
    "P-BOOKS-3021": {"max_days": 14, "conditions": ["new"]}
}

In [9]:

# Tool implementation functions to handle actual tool calls
def process_tool_call(tool_name, tool_input):
    """Process a tool call based on the tool name and input."""
    print(f"Processing tool call: {tool_name}")
    print(f"Tool input: {json.dumps(tool_input, indent=2)}")

    # Customer lookup
    if tool_name == "get_customer":
        customer_id = tool_input.get("customer_id")
        if customer_id in CUSTOMER_DB:
            result = CUSTOMER_DB[customer_id]
            print(f"Found customer: {customer_id}")
            return result
        print(f"Customer not found: {customer_id}")
        return {"error": "Customer not found"}

    # Order lookup
    elif tool_name == "get_order":
        order_id = tool_input.get("order_id")
        if order_id in ORDER_DB:
            result = ORDER_DB[order_id]
            print(f"Found order: {order_id}")
            # Calculate days since purchase for convenience
            order_date = datetime.datetime.strptime(result["date"], "%Y-%m-%d")
            today = datetime.datetime.now()
            days_since = (today - order_date).days
            print(f"Days since purchase: {days_since}")
            # Add this info to the result for convenience
            result["days_since_purchase"] = days_since
            return result
        print(f"Order not found: {order_id}")
        return {"error": "Order not found"}

    # Return eligibility check
    elif tool_name == "check_return_eligibility":
        product_id = tool_input.get("product_id")
        days = tool_input.get("days_since_purchase")
        condition = tool_input.get("product_condition")

        print(f"Checking return eligibility for product: {product_id}")
        print(f"Days since purchase: {days}")
        print(f"Product condition: {condition}")

        if product_id not in PRODUCT_RETURN_POLICIES:
            print(f"Product not found in return policy database: {product_id}")
            return {"eligible": False, "reason": "Product not found in return policy database"}

        policy = PRODUCT_RETURN_POLICIES[product_id]
        print(f"Return policy for product: max_days={policy['max_days']}, conditions={policy['conditions']}")

        if days > policy["max_days"]:
            print(f"Return window expired: {days} days > {policy['max_days']} days")
            return {
                "eligible": False,
                "reason": f"Return window of {policy['max_days']} days has passed"
            }

        if condition not in policy["conditions"]:
            print(f"Condition not eligible: {condition} not in {policy['conditions']}")
            return {
                "eligible": False,
                "reason": f"Product condition '{condition}' is not eligible for return"
            }

        print("Product is eligible for return")
        return {"eligible": True, "max_refund_percentage": 100 if condition == "new" else 70}

    # Process return
    elif tool_name == "process_return":
        # In a real implementation, this would actually update databases
        # Here we just simulate success
        order_id = tool_input.get("order_id")
        product_id = tool_input.get("product_id")
        print(f"Processing return for order: {order_id}, product: {product_id}")

        reference = "RET-" + order_id + "-" + product_id[-4:]
        print(f"Return successfully processed with reference: {reference}")

        return {
            "success": True,
            "reference_number": reference,
            "estimated_processing_time": "3-5 business days"
        }

    # The "think" tool - nothing to process, just acknowledge
    elif tool_name == "think":
        # We don't need to do anything with the "thought" input
        # It's just Claude's notes to itself
        thought = tool_input.get("thought", "")
        print("\n" + "="*80)
        print("📝 CLAUDE'S THOUGHT PROCESS:")
        print("-"*80)
        print(f"{thought}")
        print("="*80 + "\n")
        print("Claude used the 'think' tool")
        return None

    else:
        print(f"Unknown tool: {tool_name}")
        return {"error": f"Unknown tool: {tool_name}"}

# System prompt with "think" tool instructions
system_prompt = """
You are a customer service assistant for an e-commerce store. Your job is to help customers with returns and refunds.

## Return Policy Rules:
- Electronics can be returned within 30 days if new or used (but not damaged)
- Clothing can be returned within 45 days if new only
- Books can be returned within 14 days if new only
- Premium customers may be granted exceptions in certain cases
- All returns must have a valid order ID and product ID

## Using the think tool
Before taking any action or responding to the user after receiving tool results, use the think tool as a scratchpad to:
- List the specific return policy rules that apply to the current request
- Check if all required information has been collected
- Verify that the planned action complies with all policies
- Iterate over tool results for correctness

Here are some examples of what to think about inside the think tool:

<think_tool_example>
User wants to return headphones from order ORD-9876
- Need to verify: customer details, order details, product condition
- Check return policies:
  * Is it an electronics item? Yes - eligible within 30 days if new or used
  * How many days since purchase? Need to calculate
  * What condition is the product in? Need to ask user
- If eligible for return:
  * What refund method does the user prefer?
  * Process the return with all required inputs
</think_tool_example>

Be thorough in your thinking to ensure the customer receives the correct information and all policies are properly followed.
"""

def handle_conversation(messages, user_message, model="claude-3-7-sonnet-20250219"):
    """
    Handle a multi-turn conversation with Claude using the think tool.

    Args:
        messages: List of message dictionaries in the Anthropic Messages API format
        user_message: The user's message as a string
        model: The Claude model to use

    Returns:
        The updated messages list with Claude's response
    """
    tools = get_tools()

    print("\n" + "="*80)
    print(f"🧑 USER MESSAGE: {user_message}")
    print("="*80)

    # Make the initial request to Claude
    print(f"Sending initial request to Claude with model: {model}")
    response = client.messages.create(
        model=model,
        max_tokens=1024,
        tools=tools,
        messages=[{"role": "user", "content": user_message}],
        system=system_prompt
    )

    print(f"Received response with stop_reason: {response.stop_reason}")

    # Process any tool uses
    iteration = 1
    while response.stop_reason == "tool_use":
        print(f"\n📊 ITERATION {iteration}: Claude is using tools")

        # Find the tool use block
        tool_use = next((block for block in response.content if isinstance(block, ToolUseBlock)), None)

        if tool_use:
            # Extract tool details
            tool_name = tool_use.name
            tool_input = tool_use.input

            print(f"🔧 TOOL USED: {tool_name}")
            if tool_name != "think":
                print(f"📥 TOOL INPUT: {json.dumps(tool_input, indent=2)}")

            # Process the tool call
            tool_result = process_tool_call(tool_name, tool_input)

            # Add the response and tool result to the messages
            messages.append({"role": "assistant", "content": response.content})

            # Format the tool result message
            if tool_result is not None:
                print(f"📤 TOOL RESULT: {json.dumps(tool_result, indent=2)}")
                tool_result_content = json.dumps(tool_result)
            else:
                tool_result_content = "Thought recorded."

            tool_result_message = [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,
                    "content": tool_result_content
                }
            ]

            messages.append({"role": "user", "content": tool_result_message})

            # Make another request to Claude with the updated messages
            print(f"Sending follow-up request to Claude (iteration {iteration})")
            response = client.messages.create(
                model=model,
                max_tokens=1024,
                tools=tools,
                messages=messages,
                system=system_prompt
            )

            print(f"Received response with stop_reason: {response.stop_reason}")
            iteration += 1
        else:
            # No tool use block found, break the loop
            print("No tool use block found in response, breaking loop")
            break

    # Extract Claude's final text response
    final_text = ""
    for block in response.content:
        if hasattr(block, "text") and block.text:
            final_text += block.text

    print("\n" + "="*80)
    print("🤖 CLAUDE'S FINAL RESPONSE:")
    print("-"*80)
    print(final_text)
    print("="*80)

    # Add Claude's final response to the messages
    messages.append({"role": "assistant", "content": response.content})

    return messages

def main():
    """Example usage of the "think" tool in a customer service scenario."""
    print("\n" + "*"*100)
    print("*** CLAUDE 'THINK' TOOL DEMONSTRATION ***")
    print("*"*100)
    print("\nThis example shows how Claude uses the 'think' tool to reason through a customer service scenario.")
    print("We'll see Claude work through a return request using multiple tools, including the 'think' tool.")

    # Start a new conversation
    messages = []

    # User's initial message
    user_message = """I bought some wireless headphones last month from your store, order number ORD-9876.
    They worked fine but I don't like the sound quality. Can I return them?"""

    # Process the conversation
    print("Starting conversation")
    updated_messages = handle_conversation(messages, user_message)

    # Show conversation summary
    print("\n" + "*"*100)
    print("*** CONVERSATION SUMMARY ***")
    print("*"*100)

    # Count tool usage
    tool_usage = {}
    for message in updated_messages:
        if message["role"] == "assistant":
            for block in message["content"]:
                if hasattr(block, "name"):
                    tool_name = block.name
                    tool_usage[tool_name] = tool_usage.get(tool_name, 0) + 1

    print("\n📊 TOOL USAGE STATISTICS:")
    for tool, count in tool_usage.items():
        print(f"  - {tool}: {count} times")

    # Show if think tool was used
    if "think" in tool_usage:
        print(f"\n🧠 The 'think' tool was used {tool_usage['think']} times")
        print("  This shows Claude pausing to reason through the complex decision process")
    else:
        print("\n⚠️ The 'think' tool was NOT used in this conversation")

    print("\n" + "*"*100)
    print("*** DEMONSTRATION COMPLETE ***")
    print("*"*100)

    print("Conversation completed")



In [10]:
main()


****************************************************************************************************
*** CLAUDE 'THINK' TOOL DEMONSTRATION ***
****************************************************************************************************

This example shows how Claude uses the 'think' tool to reason through a customer service scenario.
We'll see Claude work through a return request using multiple tools, including the 'think' tool.
Starting conversation
Loading tool definitions

🧑 USER MESSAGE: I bought some wireless headphones last month from your store, order number ORD-9876.
    They worked fine but I don't like the sound quality. Can I return them?
Sending initial request to Claude with model: claude-3-7-sonnet-20250219


2025-03-23 19:58:27 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Received response with stop_reason: tool_use

📊 ITERATION 1: Claude is using tools
🔧 TOOL USED: get_order
📥 TOOL INPUT: {
  "order_id": "ORD-9876"
}
Processing tool call: get_order
Tool input: {
  "order_id": "ORD-9876"
}
Found order: ORD-9876
Days since purchase: 50
📤 TOOL RESULT: {
  "customer_id": "C123456",
  "date": "2025-02-01",
  "items": [
    {
      "product_id": "P-ELECTRONICS-1001",
      "name": "Wireless Headphones",
      "price": 149.99,
      "quantity": 1
    },
    {
      "product_id": "P-CLOTHING-5032",
      "name": "Premium Cotton T-shirt",
      "price": 29.99,
      "quantity": 2
    }
  ],
  "total": 209.97,
  "days_since_purchase": 50
}
Sending follow-up request to Claude (iteration 1)


2025-03-23 19:58:29 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Received response with stop_reason: tool_use

📊 ITERATION 2: Claude is using tools
🔧 TOOL USED: get_customer
📥 TOOL INPUT: {
  "customer_id": "C123456"
}
Processing tool call: get_customer
Tool input: {
  "customer_id": "C123456"
}
Found customer: C123456
📤 TOOL RESULT: {
  "name": "Jane Smith",
  "email": "jane.smith@example.com",
  "type": "premium",
  "since": "2023-01-15"
}
Sending follow-up request to Claude (iteration 2)


2025-03-23 19:58:35 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Received response with stop_reason: tool_use

📊 ITERATION 3: Claude is using tools
🔧 TOOL USED: think
Processing tool call: think
Tool input: {
  "thought": "Let me analyze this return request:\n\n1. Product Information:\n   - Product: Wireless Headphones (P-ELECTRONICS-1001)\n   - Category: Electronics (based on product ID)\n   - Purchase date: February 1, 2025\n   - Days since purchase: 50 days\n\n2. Return Policy for Electronics:\n   - Electronics can be returned within 30 days if new or used (but not damaged)\n   - This is past the standard 30-day window for electronics\n\n3. Customer Information:\n   - Customer: Jane Smith (C123456)\n   - Customer type: Premium\n   - Premium customers may be granted exceptions in certain cases\n\n4. Missing Information:\n   - Product condition (new, used, or damaged) - need to know this\n   - Reason for return - required for processing\n   - Preferred refund method - required for processing\n\n5. Next Steps:\n   - The return is technically past th

2025-03-23 19:58:39 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Received response with stop_reason: end_turn

🤖 CLAUDE'S FINAL RESPONSE:
--------------------------------------------------------------------------------
I see you're interested in returning wireless headphones from order ORD-9876. I've checked your order details and noticed a few things:

1. The wireless headphones were purchased 50 days ago on February 1, 2025
2. Our standard return policy for electronics is 30 days

I notice you're a premium customer, Jane. In some cases, we can make exceptions for our premium members. To help me determine if we can process this return, could you please tell me:

1. What is the condition of the headphones (new, used, or damaged)?
2. What is the reason you'd like to return them?
3. If eligible for return, what refund method would you prefer (original payment, store credit, or exchange)?

Once I have this information, I'll be able to better assist you with your return request.

**************************************************************************